# Sesión 9B

## Introducción a redes bayesianas

> **Objetivos:**
> - Aprender qué es una red Bayesiana y cómo se factorizan distribuciones sobre ellas.

### _¿Qué son los modelos gráficos probabilísticos?_

Los MGPs son un _marco general_ para representar y razonar sobre incertidumbre en sistemas complejos.

El nombre tiene tres palabras clave: **modelos**, **gráficos** y **probabilísticos**.

#### Modelos

* Representación declarativa de cómo entendemos el mundo.
* Permite separar:
    * La construcción del modelo (por humano o mediante aprendizaje automático).
    * De los _algoritmos de inferencia_ (que responden preguntas sobre el modelo).
    * De los _métodos de aprendizaje_ (que ajustan sus parámetros con datos).

> Un "modelo" gráfico probabilístico describe cómo creemos que las cosas se relacionan, sin importar todavía cómo las calculamos.

#### Probabilísticos

Se llama así porque los modelos tratan con **incertidumbre**.

> "Probabilístico" significa que el modelo admite la duda y razona con ella de forma lógica

#### Gráficos

Es `gráfico` porque usamos grafos (nodos y aristas) para representar las dependencias entre variables.

* Cada **nodo** representa una **variable aleatoria**.
* Cada **arista** indica una **relación probabilística (una dependencia directa)**.

Estos grafos permiten:

* Representar distribuciones gigantes de forma compacta (sin enumerar todas las combinaciones posibles).
* Razonar, usando la estructura del grafo para decidir qué variables influyen en cuáles.
* Aprender sus parámetros con pocos datos o cinluso con ayuda de expertos humanos.

> "Gráfico" significa que el modelo usa conexiones visuales (nodos y aristas) para representar dependencias probabilísticas entre variables.

## 1. Preliminares

### 1.1. Ejemplo: Modelo de estudiante

Consideramos el caso de un **estudiante dentro de un curso**.

Queremos razonar acerca de las siguientes variables aleatorias:

- **I** → Inteligencia del estudiante  
- **D** → Dificultad del curso  
- **C** → Calificación del estudiante dentro del curso  
- **P** → Puntaje en las pruebas estatales  
- **R** → Carta de recomendación laboral emitida por el profesor


#### Discretización de variables

**``Inteligencia (I)``**

$$
\mathrm{Val}(I) = \{ i^0, i^1 \}
$$  

  - $i^0$: inteligencia **baja**  
  - $i^1$: inteligencia **alta**


**``Dificultad (D)``**  

$$
\mathrm{Val}(D) = \{ d^0, d^1 \}
$$  

  - $d^0$: curso **fácil**  
  - $d^1$: curso **difícil**


**``Calificación (C)``** 

$$
\mathrm{Val}(C) = \{ c^0, c^1, c^2 \}
$$  

  - $c^0$: **baja** ($C < 6$)  
  - $c^1$: **media** ($6 \leq C < 9$)  
  - $c^2$: **alta** ($C \geq 9$)

**``Puntaje de examen (E)``**

$$
\mathrm{Val}(E) = \{ e^0, e^1 \}
$$  

  - $e^0$: **mal puntaje**  
  - $e^1$: **buen puntaje**

**``Carta de recomendación (R)``** 

$$
\mathrm{Val}(R) = \{ r^0, r^1 \}
$$  

  - $r^0$: carta **débil**  
  - $r^1$: carta **fuerte**

#### ❓ Pregunta

De no usar **modelos gráficos probabilísticos**, _¿cuántos parámetros necesitaríamos para especificar por completo la distribución sobre las cinco variables mencionadas($I, D, C, E, R$)?_

<details>
<summary>Respuesta</summary>

Cada variable puede tomar un cierto número de valores:

$$
|\mathrm{Val}(I)| = 2, \quad
|\mathrm{Val}(D)| = 2, \quad
|\mathrm{Val}(C)| = 3, \quad
|\mathrm{Val}(P)| = 2, \quad
|\mathrm{Val}(R)| = 2
$$

Por tanto, el número total de combinaciones posibles es:

$$
2 \times 2 \times 3 \times 2 \times 2 = 48
$$
 
> Son parámetros **independientes** aquellos cuyo valor **no está completamente determinado** por el valor de otros parámetros.

En este caso, la distribución $P(I,D,C,E,R)$ se especifica con **48 parámetros**, sin embargo, si hablamos de parámetros independientes:

$$
48 - 1 = 47
$$

Si no utilizáramos modelos gráficos probabilísticos, necesitaríamos **47 parámetros independientes** para especificar completamente la distribución conjunta sobre las variables $I, D, C, |, R$.
</details>

#### Red Bayesiana 

Proponemos la siguiente estructura:
- La **inteligencia** $I$ y la **dificultad** $D$ causan la **calificación** $C$.
- La **inteligencia** $I$ influye en el **puntaje** $E$ de pruebas estandarizadas.
- La **calificación** $C$ influye en la **recomendación** $R$.


```{figure} ../images/sesion9-student-model.png
:alt: student-model
:fig-align: center
:width: 700px
```

Al final de esta clase, entenderemos **cómo codificar la distribución de las variables** sobre esta red bayesiana y, a partir de ello, **por qué este modelo necesita muchos menos parámetros** que el modelo genérico (la **distribución conjunta completa**).

### 1.2. Distribuciones de probabilidad y sus operaciones

Consideremos el ejemplo del estudiante, esta vez reducido a las variables $I$, $D$ y $C$.

Una distribución conjunta, $P(I,D,C)$, sobre estas tres variables es:

|  $I$  |  $D$  |  $C$  |  $P$  |
| ----- | ----- | ----- | ----- |
| $i^0$ | $d^0$ | $c^0$ | 0.126 |
| $i^0$ | $d^0$ | $c^1$ | 0.168 |
| $i^0$ | $d^0$ | $c^2$ | 0.126 |
| $i^0$ | $d^1$ | $c^0$ | 0.126 |
| $i^0$ | $d^1$ | $c^1$ | 0.045 |
| $i^0$ | $d^1$ | $c^2$ | 0.009 |
| $i^1$ | $d^0$ | $c^0$ | 0.0056|
| $i^1$ | $d^0$ | $c^1$ | 0.0224|
| $i^1$ | $d^0$ | $c^2$ | 0.252 |
| $i^1$ | $d^1$ | $c^0$ | 0.024 |
| $i^1$ | $d^1$ | $c^1$ | 0.036 |
| $i^1$ | $d^1$ | $c^2$ | 0.06  |

**Pregunta.** ¿Cuántos parámetros en total?


<details>
<summary>Respuesta</summary>

$$2 \times 2 \times 3 = 12$$

</details>

In [1]:
# Importamos pgmpy.factors.discrete.JointProbabilityDistribution
from pgmpy.factors.discrete import JointProbabilityDistribution
from pgmpy.factors.discrete import DiscreteFactor

In [2]:
JointProbabilityDistribution?

Init signature: JointProbabilityDistribution(variables, cardinality, values)
Docstring:      Base class for Joint Probability Distribution
Init docstring:
Initialize a Joint Probability Distribution class.

Defined above, we have the following mapping from variable
assignments to the index of the row vector in the value field:

+-----+-----+-----+-------------------------+
|  x1 |  x2 |  x3 |    P(x1, x2, x2)        |
+-----+-----+-----+-------------------------+
| x1_0| x2_0| x3_0|    P(x1_0, x2_0, x3_0)  |
+-----+-----+-----+-------------------------+
| x1_1| x2_0| x3_0|    P(x1_1, x2_0, x3_0)  |
+-----+-----+-----+-------------------------+
| x1_0| x2_1| x3_0|    P(x1_0, x2_1, x3_0)  |
+-----+-----+-----+-------------------------+
| x1_1| x2_1| x3_0|    P(x1_1, x2_1, x3_0)  |
+-----+-----+-----+-------------------------+
| x1_0| x2_0| x3_1|    P(x1_0, x2_0, x3_1)  |
+-----+-----+-----+-------------------------+
| x1_1| x2_0| x3_1|    P(x1_1, x2_0, x3_1)  |
+-----+-----+-----+-------

In [3]:
p_IDC = JointProbabilityDistribution(
    variables=['I', 'D', 'C'],
    cardinality=[2, 2, 3],
    values=[0.126, 0.168, 0.126, 0.126, 0.045, 0.009, 0.0056, 0.0224, 0.252, 0.024, 0.036, 0.06]
)

In [4]:
print(p_IDC)

+------+------+------+------------+
| I    | D    | C    |   P(I,D,C) |
+======+======+======+============+
| I(0) | D(0) | C(0) |     0.1260 |
+------+------+------+------------+
| I(0) | D(0) | C(1) |     0.1680 |
+------+------+------+------------+
| I(0) | D(0) | C(2) |     0.1260 |
+------+------+------+------------+
| I(0) | D(1) | C(0) |     0.1260 |
+------+------+------+------------+
| I(0) | D(1) | C(1) |     0.0450 |
+------+------+------+------------+
| I(0) | D(1) | C(2) |     0.0090 |
+------+------+------+------------+
| I(1) | D(0) | C(0) |     0.0056 |
+------+------+------+------------+
| I(1) | D(0) | C(1) |     0.0224 |
+------+------+------+------------+
| I(1) | D(0) | C(2) |     0.2520 |
+------+------+------+------------+
| I(1) | D(1) | C(0) |     0.0240 |
+------+------+------+------------+
| I(1) | D(1) | C(1) |     0.0360 |
+------+------+------+------------+
| I(1) | D(1) | C(2) |     0.0600 |
+------+------+------+------------+


In [6]:
# Verificar que la distribución es válida
p_IDC.values.sum()

np.float64(1.0)

In [ ]:
isinstance(p_IDC, JointProbabilityDistribution), isinstance(p_IDC, DiscreteFactor)

| Clase | Qué representa | Ejemplo |
| :---- | :------------- | :------ |
| `DiscreteFactor` | Función genérica sobre variables discretas (no necesariamente normalizada) | $\phi(X, Y, Z)$ |
| `JointProbabilityDistribution` | Caso especial de `DiscreteFactor` donde la función es una **probabilidad válida** (suma = 1) | $P(X, Y, Z)$ |

> Toda `JointProbabilityDistribution` **es** un `DiscreteFactor`, pero no todo `DiscreteFactor` **es** una `JointProbabilityDistribution`.

### ¿Qué operaciones podemos llevar a cabo sobre una distribucción?

**1. Reducción**

Supongamos que observamos que la calificación final del estudiante es alta, esto es, $C=c^2$. La operación de *reducción* consiste en eliminar todas las filas que no son consistentes con la observación:

In [7]:
p_IDC.reduce?

Signature: p_IDC.reduce(values, inplace=True, show_warnings=True)
Docstring:
Reduces the factor to the context of given variable values. The variables which
are reduced would be removed from the factor.

Parameters
----------
values: list, array-like
    A list of tuples of the form (variable_name, variable_state).

inplace: boolean
    If inplace=True it will modify the factor itself, else would return
    a new factor.

show_warnings: boolean
    Whether to show warning when state name not found.

Returns
-------
Reduced factor: pgmpy.factors.discrete.DiscreteFactor or None
    If inplace=True (default) returns None else returns a new `DiscreteFactor` instance.

Examples
--------
>>> from pgmpy.factors.discrete import DiscreteFactor
>>> phi = DiscreteFactor(['x1', 'x2', 'x3'], [2, 3, 2], range(12))
>>> phi.reduce([('x1', 0), ('x2', 0)])
>>> phi.variables
['x3']
>>> phi.cardinality
array([2])
>>> phi.values
array([0., 1.])
File:      d:\anaconda3\envs\mgp\lib\site-packages\pgmpy\facto

In [8]:
# Operación de reducción C=c2
p_IDC_reduce_c2 = p_IDC.reduce(values=[('C', 2)], inplace=False)

In [9]:
print(p_IDC_reduce_c2)

+------+------+----------+
| I    | D    |   P(I,D) |
+======+======+==========+
| I(0) | D(0) |   0.1260 |
+------+------+----------+
| I(0) | D(1) |   0.0090 |
+------+------+----------+
| I(1) | D(0) |   0.2520 |
+------+------+----------+
| I(1) | D(1) |   0.0600 |
+------+------+----------+


In [10]:
# Verificar si es una distribución válida
p_IDC_reduce_c2.values.sum()

np.float64(0.447)

Matemáticamente, esta operación equivale a considerar la distribución evaluada

$$P(I, D, C=c^2) = P(I, D, c^2)$$

**Pregunta**. ¿Es este resultado una distribución de probabilidad sobre las variables $I,D$?


*Respuesta:* No.

**2. Condición**

A partir de la operación de **reducción**, si queremos obtener una distribución legítima sobre las variables que no reducimos, debemos dividir sobre la suma:

In [11]:
p_IDC.conditional_distribution?

Signature: p_IDC.conditional_distribution(values, inplace=True)
Docstring:
Returns Conditional Probability Distribution after setting values to 1.

Parameters
----------
values: list or array_like
    A list of tuples of the form (variable_name, variable_state).
    The values on which to condition the Joint Probability Distribution.
inplace: Boolean (default True)
    If False returns a new instance of JointProbabilityDistribution

Examples
--------
>>> import numpy as np
>>> from pgmpy.factors.discrete import JointProbabilityDistribution
>>> prob = JointProbabilityDistribution(['x1', 'x2', 'x3'], [2, 2, 2], np.ones(8)/8)
>>> prob.conditional_distribution([('x1', 1)])
>>> print(prob)
x2    x3      P(x2,x3)
----  ----  ----------
x2_0  x3_0      0.2500
x2_0  x3_1      0.2500
x2_1  x3_0      0.2500
x2_1  x3_1      0.2500
File:      d:\anaconda3\envs\mgp\lib\site-packages\pgmpy\factors\discrete\jointprobabilitydistribution.py
Type:      method

In [12]:
# Operación de condición sobre C=c2
p_IDC_cond_c2 = p_IDC.conditional_distribution([('C', 2)], inplace=False)

In [13]:
print(p_IDC_cond_c2)

+------+------+----------+
| I    | D    |   P(I,D) |
+======+======+==========+
| I(0) | D(0) |   0.2819 |
+------+------+----------+
| I(0) | D(1) |   0.0201 |
+------+------+----------+
| I(1) | D(0) |   0.5638 |
+------+------+----------+
| I(1) | D(1) |   0.1342 |
+------+------+----------+


In [14]:
# Verificar si es una distribución válida
p_IDC_cond_c2.values.sum()

np.float64(1.0)

In [15]:
# Esto es equivalente a la operación de reducción C=c2, y posteriormente normalizar
p_IDC_reduce_c2.values / p_IDC_cond_c2.values

array([[0.447, 0.447],
       [0.447, 0.447]])

Matemáticamente, esta operación equivale a considerar la distribución condicionada $P(I, D| C=c^2) = P(I, D| c^2)$.

**Pregunta**. ¿Es este resultado una distribución de probabilidad sobre las variables $I,D$?

*Respuesta:* Sí.

**3. Marginalización**

Cuando tenemos una distribución de probabilidad sobre un conjunto de variables, y producimos una sobre un subconjunto de las variables originales. Por ejemplo, queremos la distribución marginal sobre $I, D$:

In [16]:
# Imprimir distribución inicial
print(p_IDC)

+------+------+------+------------+
| I    | D    | C    |   P(I,D,C) |
+======+======+======+============+
| I(0) | D(0) | C(0) |     0.1260 |
+------+------+------+------------+
| I(0) | D(0) | C(1) |     0.1680 |
+------+------+------+------------+
| I(0) | D(0) | C(2) |     0.1260 |
+------+------+------+------------+
| I(0) | D(1) | C(0) |     0.1260 |
+------+------+------+------------+
| I(0) | D(1) | C(1) |     0.0450 |
+------+------+------+------------+
| I(0) | D(1) | C(2) |     0.0090 |
+------+------+------+------------+
| I(1) | D(0) | C(0) |     0.0056 |
+------+------+------+------------+
| I(1) | D(0) | C(1) |     0.0224 |
+------+------+------+------------+
| I(1) | D(0) | C(2) |     0.2520 |
+------+------+------+------------+
| I(1) | D(1) | C(0) |     0.0240 |
+------+------+------+------------+
| I(1) | D(1) | C(1) |     0.0360 |
+------+------+------+------------+
| I(1) | D(1) | C(2) |     0.0600 |
+------+------+------+------------+


In [17]:
# Marginalizar I, D
p_IDC_marg_ID = p_IDC.marginalize(variables=['I', 'D'], inplace=False)

In [18]:
print(p_IDC_marg_ID)

+------+--------+
| C    |   P(C) |
+======+========+
| C(0) | 0.2816 |
+------+--------+
| C(1) | 0.2714 |
+------+--------+
| C(2) | 0.4470 |
+------+--------+


Y si queremos la marginal sobre $I$ nada más:

In [19]:
# Marginalizar C y D
p_IDC_marg_CD = p_IDC.marginalize(variables=['C', 'D'], inplace=False)

In [20]:
print(p_IDC_marg_CD)

+------+--------+
| I    |   P(I) |
+======+========+
| I(0) | 0.6000 |
+------+--------+
| I(1) | 0.4000 |
+------+--------+


In [21]:
# Marginalizar C y I
p_IDC_marg_CI = p_IDC.marginalize(variables=['C', 'I'], inplace=False)

In [22]:
print(p_IDC_marg_CI)

+------+--------+
| D    |   P(D) |
+======+========+
| D(0) | 0.7000 |
+------+--------+
| D(1) | 0.3000 |
+------+--------+


Matemáticamente, las anteriores operaciones equivalen a:

$$P(I, D) = \sum_{c\in\mathrm{Val}(C)} P(I, D, C=c), \text{ y }$$

$$P(I) = \sum_{(c,d)\in\mathrm{Val}(C,D)} P(I, D=d, C=c).$$

En abuso de la notación, para no hacer engorrosa la escritura, las anteriores sumas se expresan comúnmente como:

$$P(I, D) = \sum_{C} P(I, D, C), \text{ y }$$

$$P(I) = \sum_{C,D} P(I, D, C).$$

## 2. Fundamentos de redes bayesianas

### 2.1. Modelando independencias

_**¿Qué necesitamos hacer para para que estos nodos y aristas representen una distribución de probabilidad?**_

```{figure} ../images/sesion9-student-model.png
:alt: student-model
:fig-align: center
:width: 700px
```

* **1. Cada nodo tiene su mini tabla de comportamiento (CPD)**

Una CPD _(Conditional Probability Distribution)_ describe cómo se comporta una variable **dado** el estado de sus padres en el grafo.

Por ejemplo:

* $P(D)$ → solo depende de sí misma (no tiene padres).

* $P(I)$ → igual, es independiente.

* $P(C \mid I,D)$ → depende de la inteligencia y la dificultad.

* $P(E \mid I)$ → depende de la inteligencia.

* $P(R \mid C)$ → depende de la calificación.

* **2. Las CPDs son los ladrillos básicos de la red**

Cada nodo de la red tiene su propia CPD -es decir, su propio "bloque de conocimiento local"-.

Si la red tiene $5$ nodos, entonces hay $5$ CPDs.

Al unirlas, no tienes una lista infinita de casos posibles, sino **pequeñas piezas de información localmente coherentes**.

* **3.¿Cómo se unen todas?**

Para obtener la **distribución conjunta completa**, multiplicamos todas las CPDs entre sí.

Esto se llama la _regla de la cadena de las redes bayesianas_.

$$
P(I,D,C,E,R) = P(I) \times P(D) \times P(C \mid I,D) \times P(E \mid I) \times P(R \mid C)
$$

💬 Intuitivamente:

>Empieza con las causas base (I, D) y ve multiplicando los efectos condicionales según el grafo.

* **4. ¿qué estamos haciendo realmente?**

Matemáticamente, lo que hacemos al multiplicar las CPDs es un **producto de factores.**

Cada CPD es un **factor** (una tabla con valores numéricos).

Las CPDs *comparten variables* (por ejemplo, $C$ aparece en dos CPDs: $P(C \mid I,D)$ y $P(R \mid C)$), así que al multiplicarlas obtenemos un **factor grande**, cuya **alcance** _scope_ incluye **todas las variables** de la red.

* **5. ¿Por qué esto es tan poderoso?**

Porque...
- No necesitas escribir $48$ números distintos.
- Solo defines unas pocas tablas locales.
- La estructura del grafo ya codifica las **independencias** entre variables.

Así, puedes reconstruir la distribución completa de una manera compacta y comprensible.

| Elemento | Significado | Ejemplo |
|-----------|--------------|----------|
| **Nodo sin padres** | Distribución simple | $P(I)$, $P(D)$ |
| **Nodo con padres** | CPD (condicional) | $P(C \mid I, D)$ |
| **Toda la red** | Producto de todas las CPDs | $P(I)P(D)P(C \mid I,D)P(E \mid I)P(R \mid C)$ |
| **Resultado final** | Distribución conjunta completa | $P(I,D,C,E,R)$ |


```{figure} ../images/sesion9-student-model-factors.png
:alt: student-model-factors
:fig-align: center
:width: 800px
```

### La justificación formal: regla de la cadena + independencia

$$
P(I, D, C, E, R) = P(I) \times P(D) \times P(C \mid I, D) \times P(E \mid I) \times P(R \mid C)
$$

Este producto no es casual: surge directamente de la **regla de la cadena de la probabilidad**, combinada con las **independencias condicionales** que nos indica la estructura del grafo.

🔹 **Paso 1: Aplicamos la regla de la cadena**

La regla de la cadena dice que cualquier distribución conjunta se puede descomponer en condicionales:

$$
P(I, D, C, E, R) = P(I, D, C, E) \, P(R \mid I, D, C, E)
$$

Esto significa:
> “La probabilidad de todo es la probabilidad de los primeros cuatro, multiplicada por la probabilidad del último dado los demás.”

🔹 **Paso 2: Descomponemos más**

Repetimos el proceso con los primeros cuatro:

$$
P(I, D, C, E) = P(I, D, C) \, P(E \mid I, D, C)
$$

Entonces:
$$
P(I, D, C, E, R) = P(I, D, C) \, P(E \mid I, D, C) \, P(R \mid I, D, C, E)
$$

🔹 **Paso 3: Aplicamos independencias condicionales**

La **estructura del grafo** nos dice quién depende de quién:

- $R$ depende **solo de $C$** → $P(R \mid I, D, C, E) = P(R \mid C)$  
- $E$ depende **solo de $I$** → $P(E \mid I, D, C) = P(E \mid I)$  
- $D$ e $I$ son **independientes** → $P(D \mid I) = P(D)$  

Sustituyendo esas simplificaciones:

$$
P(I, D, C, E, R) = P(I, D, C) \, P(E \mid I) \, P(R \mid C)
$$

🔹 **Paso 4: Descomponemos $P(I, D, C)$**

Usando la cadena otra vez:

$$
P(I, D, C) = P(I, D) \, P(C \mid I, D)
$$

Y como $I$ y $D$ son independientes:

$$
P(I, D) = P(I) \, P(D)
$$

Entonces:

$$
P(I, D, C) = P(I) \, P(D) \, P(C \mid I, D)
$$

🔹 **Paso 5: Sustituyendo todo**

Finalmente obtenemos la **factorización de la red bayesiana**:

$$
P(I, D, C, E, R) = P(I) \, P(D) \, P(C \mid I, D) \, P(E \mid I) \, P(R \mid C)
$$

Por ejemplo,

$$P(i^1, d^0, c^0, e^1, r^1)=P(i^1) P(d^0) P(c^0 | i^1, d^0) P(e^1 | i^1) P(r^1 | c^0) = 0.3 \times 0.6 \times 0.02 \times 0.8 \times 0.01 $$

| Paso | Expresión | Interpretación |
|------|------------|----------------|
| 1 | $P(i^1)$ | Ser inteligente |
| 2 | $P(d^0)$ | Curso fácil |
| 3 | $P(c^0 \mid i^1, d^0)$ | Calificación baja aunque es inteligente |
| 4 | $P(e^1 \mid i^1)$ | Buen puntaje en la prueba |
| 5 | $P(r^1 \mid c^0)$ | Carta buena a pesar de nota baja |

Cada fila representa una **pieza local de información (una CPD)** dentro de la red bayesiana.

Al multiplicarlas todas, obtienes la probabilidad conjunta del escenario $(i^1, d^0, c^0, e^1, r^1)$

```{admonition} 🔹 Definición forma de una red bayesiana
:class: tip

Hasta ahora hemos visto las redes bayesianas de forma intuitiva: como **grafos donde cada nodo representa una variable** y **cada flecha una relación de dependencia**. 

Ahora podemos formalizar esta idea.

**Definición**

Una **red bayesiana** es un **grafo dirigido acíclico (DAG)** $\mathcal{G}$, donde:

- Cada nodo representa una **variable aleatoria** $X_i$.
- Cada nodo se asocia con una **distribución condicional**  
  $P(X_i \mid Pa_\mathcal{G}(X_i))$, donde $Pa_\mathcal{G}(X_i)$ son los **padres** de $X_i$ en el grafo.
- Cada arco indica una **influencia causal o probabilística directa**.

La **distribución conjunta** sobre todas las variables se obtiene multiplicando todas las distribuciones locales:

$$
P(X_1, X_2, \dots, X_n) = \prod_{i=1}^{n} P(X_i \mid Pa_\mathcal{G}(X_i))
$$
```

```{admonition} 🔸 Independencias locales
:class: tip

Una de las ideas más potentes de las redes bayesianas es que **la estructura del grafo ya codifica independencias**.
 
Formalmente, para cada variable $X_i$:

$$
I_l(\mathcal{G}) = \{ (X_i \perp NoDescendientes_\mathcal{G}(X_i) \mid Pa_\mathcal{G}(X_i)) \}
$$

Esto significa que, **dado sus padres**, una variable es independiente de todos los nodos que:
- no son sus padres, y  
- no son sus descendientes.

En palabras simples...

> Cada nodo “mira solo hacia arriba” (a sus padres).  
> Todo lo que esté más lejos en el grafo **no le afecta directamente** una vez que conoce el estado de sus padres.

Por eso podemos descomponer toda la distribución como un **producto de términos locales**,  
y de ahí viene el poder de las redes bayesianas: **reducen exponencialmente la complejidad** al aprovechar las independencias condicionales.
```

#### Actividad

1. Obtener las independencias locales que codifica la red Bayesiana del estudiante.

2. Comparar el número de parámetros independientes que necesita la red Bayesiana del estudiante contra el número de parámetros que necesitaría la distribución conjunta sin ninguna suposición de independencia.

**¿Cómo declarar una red Bayesiana en pgmpy?**

In [23]:
# Importamos pgmpy.models.BayesianNetwork
# Importamos pgmpy.factors.discrete.TabularCPD

from pgmpy.models import BayesianNetwork, DiscreteBayesianNetwork
from pgmpy.factors.discrete import TabularCPD

In [24]:
DiscreteBayesianNetwork?

Init signature:
DiscreteBayesianNetwork(
    ebunch=None,
    latents=set(),
    lavaan_str=None,
    dagitty_str=None,
)
Docstring:     
Initializes a Bayesian Network.
A models stores nodes and edges with conditional probability
distribution (cpd) and other attributes.

Models hold directed edges.  Self loops are not allowed neither
multiple (parallel) edges.

Nodes can be any hashable python object.

Edges are represented as links between nodes.

Parameters
----------
ebunch: input graph
    Data to initialize graph.  If ebunch=None (default) an empty
    graph is created.  The ebunch can be an edge list, or any
    NetworkX graph object.

latents: list, array-like
    List of variables which are latent (i.e. unobserved) in the model.

Examples
--------
Create an empty Bayesian Network with no nodes and no edges.

>>> from pgmpy.models import DiscreteBayesianNetwork
>>> G = DiscreteBayesianNetwork()

G can be grown in several ways.

**Nodes:**

Add one node at a time:

>>> G.add_nod

In [25]:
student_model = DiscreteBayesianNetwork(
    [("D", "C"), ("I", "C"), ("I", "E"), ("C", "R")]
)

In [26]:
TabularCPD?

Init signature:
TabularCPD(
    variable,
    variable_card,
    values,
    evidence=None,
    evidence_card=None,
    state_names={},
)
Docstring:     
Defines the conditional probability distribution table (CPD table)

Parameters
----------
variable: int, string (any hashable python object)
    The variable whose CPD is defined.

variable_card: integer
    Cardinality/no. of states of `variable`

values: 2D array, 2D list or 2D tuple
    Values for the CPD table. Please refer the example for the
    exact format needed.

evidence: array-like
    List of variables in evidences(if any) w.r.t. which CPD is defined.

evidence_card: array-like
    cardinality/no. of states of variables in `evidence`(if any)

state_names: dict (default: dict())
    A dictionary of the form {variable: list of states} specifying the
    names of possible states for each variable (variable + evidence) in
    the TabularCPD. The order in which the states are specified should
    match the order in the values 

In [27]:
# Definimos distribución condicional de D
cpd_D = TabularCPD(
    variable='D',
    variable_card=2,
    values=[
        [0.7],
        [0.3]
    ]
)
# Definimos distribución condicional de I
cpd_I = TabularCPD(
    variable='I',
    variable_card=2,
    values=[
        [0.6],
        [0.4]
    ]
)

In [28]:
print(cpd_D)

+------+-----+
| D(0) | 0.7 |
+------+-----+
| D(1) | 0.3 |
+------+-----+


La representación de las distribuciones condicionales en `pgmpy` es un poquito distinto a como está en el dibujo de arriba. En `pgmpy` las columnas representan evidencia, y las filas los distintos estados de la variable en la distribución condicional:

|         | $i^0 d^0$ | $i^0 d^1$ | $i^1 d^0$ | $i^1 d^1$ |
| ------- | --------- | --------- | --------- | --------- |
| $c^0$   | 0.3       | 0.7       | 0.02      | 0.2       |
| $c^1$   | 0.4       | 0.25      | 0.08      | 0.3       |
| $c^2$   | 0.3       | 0.05      | 0.9       | 0.5       |

In [29]:
# Definimos distribución condicional de C
cpd_C = TabularCPD(
    variable='C',
    variable_card=3,
    values=[
        [0.30, 0.70, 0.02, 0.20],
        [0.40, 0.25, 0.08, 0.30],
        [0.30, 0.05, 0.90, 0.50]
    ],
    evidence=['I', 'D'],
    evidence_card=[2, 2]
)
# Definimos distribución condicional de E
cpd_E = TabularCPD(
    variable='E',
    variable_card=2,
    values=[
        [0.95, 0.20],
        [0.05, 0.80]
    ],
    evidence=['I'],
    evidence_card=[2]
)
# Definimos distribución condicional de R
cpd_R = TabularCPD(
    variable='R',
    variable_card=2,
    values=[
        [0.99, 0.40, 0.10],
        [0.01, 0.60, 0.90]
    ],
    evidence=['C'],
    evidence_card=[3]
)

In [30]:
print(cpd_R)

+------+------+------+------+
| C    | C(0) | C(1) | C(2) |
+------+------+------+------+
| R(0) | 0.99 | 0.4  | 0.1  |
+------+------+------+------+
| R(1) | 0.01 | 0.6  | 0.9  |
+------+------+------+------+


In [31]:
# add_cpds method
student_model.add_cpds?

Signature: student_model.add_cpds(*cpds)
Docstring:
Add CPD (Conditional Probability Distribution) to the Bayesian Model.

Parameters
----------
cpds  :  list, set, tuple (array-like)
    List of CPDs which will be associated with the model

Examples
--------
>>> from pgmpy.models import DiscreteBayesianNetwork
>>> from pgmpy.factors.discrete.CPD import TabularCPD
>>> student = DiscreteBayesianNetwork([('diff', 'grades'), ('aptitude', 'grades')])
>>> grades_cpd = TabularCPD('grades', 3, [[0.1,0.1,0.1,0.1,0.1,0.1],
...                                       [0.1,0.1,0.1,0.1,0.1,0.1],
...                                       [0.8,0.8,0.8,0.8,0.8,0.8]],
...                         evidence=['diff', 'aptitude'], evidence_card=[2, 3],
...                         state_names={'grades': ['gradeA', 'gradeB', 'gradeC'],
...                                      'diff': ['easy', 'hard'],
...                                      'aptitude': ['low', 'medium', 'high']})
>>> student.add_cpds(grades_c

In [32]:
# Asociamos las distribuciones condicionales a la red
student_model.add_cpds(cpd_D, cpd_I, cpd_C, cpd_E, cpd_R)

In [33]:
# Check the model for various errors. This method checks for the following errors:  
# * Checks if the sum of the probabilities for each state is equal to 1 (tol=0.01).
# * Checks if the CPDs associated with nodes are consistent with their parents.
student_model.check_model()

True

Una vez tenemos el modelo, podemos hacer varias cosas con él. Entre ellas, podemos verificar las independencias locales que codifica el modelo:

In [34]:
# Verificar las independencias locales que implica el modelo
student_model.local_independencies('D')

(D ⟂ E, I)

In [35]:
student_model.local_independencies('C')

(C ⟂ E | D, I)

In [36]:
student_model.local_independencies?

Signature: student_model.local_independencies(variables)
Docstring:
Returns an instance of Independencies containing the local independencies
of each of the variables.

Parameters
----------
variables: str or array like
    variables whose local independencies are to be found.

Examples
--------
>>> from pgmpy.base import DAG
>>> student = DAG()
>>> student.add_edges_from([('diff', 'grade'), ('intel', 'grade'),
>>>                         ('grade', 'letter'), ('intel', 'SAT')])
>>> ind = student.local_independencies('grade')
>>> ind
(grade ⟂ SAT | diff, intel)
File:      d:\anaconda3\envs\mgp\lib\site-packages\pgmpy\base\dag.py
Type:      method

In [37]:
student_model.local_independencies(["D", "I", "C", "E", "R"])

(D ⟂ E, I)
(I ⟂ D)
(C ⟂ E | D, I)
(E ⟂ R, D, C | I)
(R ⟂ D, I, E | C)